<hr style="height:5px;border-width:2;color:gray;background-color:#000000">
<center><h1>CS 144 - Winter 2024 - Linear-Time Construction of Suffix Arrays</h1></center>
<center><h1>Due: Sunday, March 17th, 2024 @ 11:59pm</h1></center>

### Enter your information below:

<div style="color: #000000;background-color: #EEEEFF">
    Your Name (submitter): Alex Camilleri <br>
    Your student ID (submitter): 862196968
<br>
<br>
<b>By submitting this notebook, I assert that the work below is my own work, completed for this course.  Except where explicitly cited, none of the portions of this notebook are duplicated from anyone else's work or my own previous work.</b>
<br>    
<br>
<b>Instruction for submissions:</B> when you have completed this project, download this .ipynb file to your computer by left-clicking on the file name, and submit to <a href="https://elearn.ucr.edu/">Canvas</A> by the deadline.
<br>
<br>
<B>Late work:</B> There is no late deadline for the final project, except for the most serious circumstances (illness, medical emergency, etc.) which have to be documented.
</div>


<hr style="height:5px;border-width:2;color:gray;background-color:#000000">
<center><h1>The Skew algorithm</h1></center>
<br>
In this project you will implement the construction of suffix arrays in linear-time. This algorithm was <B>not</B> explained in class. Part of this project is to understand how the algorithm works before starting the implementation.  Here is some resources:
<UL>
    <li><A HREF="https://www.cs.cmu.edu/~guyb/realworld/papersS04/KaSa03.pdf">The paper in which the algorithm was originally proposed (includes C code)</A></li>
    <li><A HREF="http://www.mi.fu-berlin.de/wiki/pub/ABI/SS13Lecture3Materials/script.pdf">Another description of the algorithm (notation heavy) with an example</A></li>
    <li><A HREF="https://gist.github.com/markormesher/59b990fba09972b4737e7ed66912e044">An example</A></li>
    <li><A HREF="https://github.com/vikasawadhiya/DC3-Algorithm/blob/main/DC3AlgorithmTutorial.pdf">Another example, developed all the way</A>
    <li><A HREF="https://www.cs.cmu.edu/~ckingsf/bioinfo-lectures/suffixarrays.pdf">Carl's slide (CMU)</A></li>
    <li><A HREF="http://rob-p.github.io/CSE549F17/lectures/Lec05.pdf">Rob's slides (Stony Brook)</A></li>
    <li><A HREF="https://mailund.dk/posts/skew-python-go/">Thomas Mailund's post on Skew in GO and Python</A>
    <li><A HREF="https://www.youtube.com/watch?v=XiuSW_mEn7g">Radix Sort Algorithm Introduction in 5 Minutes</A></li>
    <li><A HREF="https://www.youtube.com/watch?v=OKd534EWcdk">Learn Counting Sort Algorithm in LESS THAN 6 MINUTES!</A></li>
    <li><A HREF="https://www.youtube.com/watch?v=T8XgD40I6uE">COMP526 6-7 §6.6 Linear-time suffix sorting</A></li>
    <li><A HREF="https://www.youtube.com/watch?v=phErzt2C9g8">COMP526 6-8 §6.6 Linear time suffix sorting (part 2)</A></li>
    <li>Use <A HREF="https://www.google.com/search?q=suffix+array+linear+time+construction">Google</A> for more</li>
</UL>

Goals:
<UL>
    <LI>Write a working (bug-free) Python3 implementation of the Skew algorithm in JupyterHub that runs in linear time</LI>
    <LI>Make sure that the output of your implementation matches the output for the naive/slow implementation that we developed in homework 3</LI>
    <LI>Collect experimental results on running your implementation for larger and larger inputs, say 1000, 2000, 5000, 10000, 100000 symbols, and plot the running time as a function of the input size</LI>
    <LI>Compare the performance of your methods against with some <A HREF="https://louisabraham.github.io/notebooks/suffix_arrays.html">fast Python implementations</A></li>
</UL>

You just need to compute the suffix array, not the LCP (longest common prefix) array.
You are allowed to study C/C++ code, but you have to write your own Python code. It is mandatory to acknowledge sources.


In [ ]:
import time

def triplet(x, i):
    return (safe_idx(x, i), safe_idx(x, i + 1), safe_idx(x, i + 2))

def rank(x, idx):   # Create rank
    rank_dict = {}
    for i in idx:
        k_string = triplet(x, i)
        if k_string not in rank_dict:
            rank_dict[k_string] = len(rank_dict) + 2      # 1 and 2 used for string separation
    #print("Rank in rank_dict:",rank_dict)
    return rank_dict

def u_idx(i, m) -> int:
    # indices in u-string are matched back to the original string - helps recursive check for duplicates
    if i < m: return 1 + 3 * i
    else: return 2 + 3 * (i - m - 1)

def less(x, i, j, ISA):
    a, b = safe_idx(x, i), safe_idx(x, j)
    if a < b: return True
    if a > b: return False
    if i % 3 != 0 and j % 3 != 0: return ISA[i] < ISA[j]
    return less(x, i + 1, j + 1, ISA)

def build_u(x, alpha):
    # Make u string - partition (1) at len(u) // 2.
        # u = S1 indexs + S2 indexs
    return [ *(alpha[triplet(x, i)] for i in range(1, len(x), 3)), 1, *(alpha[triplet(x, i)] for i in range(2, len(x), 3)) ]

def safe_idx(x, i):
    return 0 if i >= len(x) else x[i] #prevents indexing past x

def char_freq(x, asize):
    # frequency
    counts = [0] * asize
    for c in x:
        counts[c] += 1
    return counts


def sum(counts):
    res, acc = [0] * len(counts), 0
    for i, k in enumerate(counts):
        res[i] = acc
        acc += k
    return res


def bucket_sort(x, asize, idx, offset = 0):              # Uses instead of counting sort, helps for detecting duplicates w/ buckets
    # idx has indices, sorted with offset and count of symbols
    sort_symbs = [safe_idx(x, i + offset) for i in idx]
    counts = char_freq(sort_symbs, asize)
    buckets = sum(counts)
    out = [None] * len(idx)
    for i in idx:
        bucket = safe_idx(x, i + offset)
        out[buckets[bucket]] = i
        buckets[bucket] += 1
    return out

def radix_sort(x, p, arr): # Bucket sort is better than count sort - organizes for duplicates - p is array size, x is range for the string
    for i in reversed(range(3)):
        arr = bucket_sort(x, p, arr, i)
    return arr

def skew(x, asize):
    #print("Skew start x:",x,"-- asize:",asize)
    SA12 = [i for i in range(len(x)) if i % 3 != 0]
    SA12 = radix_sort(x, asize, SA12)
    rank_dict = rank(x, SA12)
    if len(rank_dict) < len(SA12):
        u = build_u(x, rank_dict)    # u-string = S1 + S2 ranks
        #print("u =",u)
        sa_u = skew(u, len(rank_dict) + 2) # +2 because 0 is used as stop char for indexing through string - 1 is used as separator - recursive skew call
        # u represents all strings from SA12 but at 2/3rds n size,

        # Then map u's suffix array back to a sorted SA12
        m = len(sa_u) // 2
        SA12 = [u_idx(i, m) for i in sa_u if i != m]

    SA3 = ([len(x) - 1] if len(x) % 3 == 1 else []) + \
          [i - 1 for i in SA12 if i % 3 == 1]
    SA3 = bucket_sort(x, asize, SA3) # If the last index refers to class 0, treat it as the smallest string - bucket sort SA3

    # Merge S12 and S3
    ISA = { SA12[i]: i for i in range(len(SA12)) }
    SA = []
    i, j = 0, 0
    while i < len(SA12) and j < len(SA3):
        if less(x, SA12[i], SA3[j], ISA):
            SA.append(SA12[i])
            i += 1
        else:
            SA.append(SA3[j])
            j += 1
    SA.extend(SA12[i:])
    SA.extend(SA3[j:])
    return SA

start = time.time()

f = open("test.txt", "r")         # open the file
text = f.readline().strip()            # text is on the first line
s = skew([ord(y) for y in text], 256) # Convert to ascii string and call skew algorithm
print(s)

end = time.time()
print(end - start)

[15, 14, 0, 1, 12, 6, 4, 2, 8, 13, 3, 7, 9, 10, 11, 5]
0.005694150924682617


Credit to https://mailund.dk/posts/skew-python-go/ for the fix for duplicate suffixes in the radix array and handling them recursively. Most code based off of both Mark Ormesher's explanation (https://gist.github.com/markormesher/59b990fba09972b4737e7ed66912e044) and Thomas Mailund's blog for skew method in linear time.|

Test in linear time for sequences created using UCR's Random DNA sequence generator by Morris F. Maduro:
1000: 0.020137310028076172,
2000: 0.03774142265319824,
5000: 0.07602787017822266,
10000: 0.141556978225708,
100000: 1.3828442096710205.

<img src="cs144-chart.png">

From https://louisabraham.github.io/articles/suffix-arrays, a chunk of the implementations for suffix arrays reach well over O(n) runtime. Some causes of this are how merge natively requires at least O(n log n) runtime. Prefix doubling takes O(n log n^2) time, and even if it is shorter, prefix matrices takes O(n log n) time. Computing the longest common prefix unfortunately doesn't produce a faster runtime. The only method that achieves a runtime of O(n) other than the skew algorithm is using suffix-tree-to-suffix-array conversion, but it has a high constant for runtime. Compared to all of these methods, while the skew method doesn't allow for simple computation, its runtime is phenominally small and allows for quick calculation of long sequences. This is due to the recursive skew call being a fraction of the string passed in, as well as computing 3-string suffixes of 'modulo 3 = 1' and 'modulo 3 = 2' for bucket sort separate from suffixes of 'modulo 3 = 0' and merging them afterwards, having a runtime of T(N) = T(2N/3) + O(n).